In [1]:
import numpy as np
from tqdm.auto import tqdm
from z3 import *
from operator import mul
from more_itertools import distinct_permutations
from functools import reduce
from collections import defaultdict

In [2]:
def z3_min(v1, v2):
    return If(v1 < v2, v1, v2)

def z3_max(v1, v2):
    return If(v1 < v2, v2, v1)

def cube_size(n, dimensions):
    return reduce(mul, [n + x for x in range(dimensions)])

def dissectable_by_cubes(dissected_cube, cubes, dimensions):
    if sum([cube_size(x, dimensions) for x in cubes]) != cube_size(dissected_cube, dimensions):
        return False
    cubes = sorted(cubes)
    if cubes[-1] > dissected_cube:
        return False
    if cubes[-1] == dissected_cube and len(cubes) > 1:
        return False
    if len(cubes) <= 1:
        return False
    if cubes[-1] + cubes[-2] > dissected_cube + dimensions - 1:
        return False
    return True

def create_answer(dissected_cube, result):
    res = np.empty((dissected_cube, dissected_cube + 1, dissected_cube + 2), dtype=int)
    for i in range(len(result)):
        r = result[i]
        for x in range(r[0][0], r[0][1]):
            for y in range(r[1][0], r[1][1]):
                for z in range(r[2][0], r[2][1]):
                    res[x, y, z] = i
    return res

def non_strict_dissection(dissected_cube, cubes, dimensions=3):
    X = np.array([[[Int(f"c{x}_d{dim}_{y}") for y in range(2)] for dim in range(dimensions)] for x in range(len(cubes))])
    X_flatten = X.reshape(-1)
    s = Solver()
    constraints = [dissected_cube + x for x in range(dimensions)]

    # Cubes should be inside the cube
    s.add([0 <= y[0] for x in X for y in x])
    s.add([y[1] <= constraints[i] for x in X for i, y in enumerate(x)])
    
    # One of possible rotations applied to the cube
    for i, cube in enumerate(cubes):
        variables = X[i]
        sides = [cube + x for x in range(dimensions)]
        rules = []
        for p in distinct_permutations(sides):
            rule = []
            for j in range(len(p)):
                rule.append(variables[j][0] + p[j] == variables[j][1])
            rules.append(And(rule))
        s.add(Or(rules))
    
    # Cubes shouldn't intersect
    for i in range(len(cubes)):
        for j in range(i + 1, len(cubes)):
            rules = []
            variables_1 = X[i]
            variables_2 = X[j]
            for dim in range(dimensions):
                m1 = z3_max(X[i][dim][0], X[j][dim][0])
                m2 = z3_min(X[i][dim][1], X[j][dim][1])
                rules.append(m1 >= m2)
            s.add(Or(rules))
    
    can_be_solved = s.check()
    if can_be_solved == sat:
        model = s.model()
        results = np.array([model.eval(x).as_long() for x in X.reshape(-1)]).reshape(X.shape)
        if dimensions == 3:
            res = create_answer(dissected_cube, results)
        else:
            res = None
        return results, res
    return None

def dissection(dissected_cube, cubes, dimensions=3):
    if not dissectable_by_cubes(dissected_cube, cubes, dimensions):
        return None
    return non_strict_dissection(dissected_cube, cubes, dimensions=dimensions)

def possible_dissections(n, dimensions, verbose=True):
    needed_size = cube_size(n, dimensions)
    cubes = np.arange(1, n)
    cube_sizes = np.array([cube_size(x, dimensions) for x in cubes])
    result = []
    prev_cubes = []
    combs = defaultdict(lambda: list())
    for i in tqdm(range(len(cubes)), leave=False, disable=not verbose):
        cur_size = cube_sizes[i]
        new_combs = defaultdict(lambda: list())
        for sz in combs:
            if sz + cur_size <= needed_size:
                new_combs[sz + cur_size].append(i)
        for sz, options in new_combs.items():
            combs[sz].extend(options)
        for j, sz in enumerate(prev_cubes):
            if sz + cur_size <= needed_size:
                combs[sz + cur_size].append((j, i))
        prev_cubes.append(cur_size)
    
    def rec_construct(sz, last=None):
        res = []
        for x in combs[sz]:
            if isinstance(x, tuple):
                if last is not None and x[1] + 1 >= last + 1:
                    continue
                res.append([x[0] + 1, x[1] + 1])
                continue
            elif last is not None and x + 1 >= last + 1:
                continue
            cur_res = rec_construct(sz - cube_sizes[x], last=x)
            for y in cur_res:
                if last is not None and y[-1] >= x + 1:
                    continue
                y.append(x + 1)
            res.extend(cur_res)
        return res
        
    sum_equal_dissections = rec_construct(needed_size)
    filtered_dissections = [x for x in sum_equal_dissections if x[-1] + x[-2] < n + dimensions]
    filtered_dissections = [x for x in filtered_dissections if non_strict_dissection(n, x[-dimensions - 2:], dimensions) is not None]
    return filtered_dissections

def all_dissections(n, dimensions=3, verbose=True):
    results = []
    p_dissections = possible_dissections(n, dimensions, verbose=verbose)
    if verbose:
        print(f"Candidate dissections: {p_dissections}")
    for x in tqdm(p_dissections, leave=False, disable=not verbose):
        res = dissection(n, x, dimensions)
        if res is not None:
            results.append((x, *res))
    return results

# Just some test
res = [x[0] for x in all_dissections(20, 2)]
res

  0%|          | 0/19 [00:00<?, ?it/s]

Candidate dissections: [[1, 2, 3, 5, 6, 7, 8, 9, 10], [4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 7, 8, 9, 11], [1, 2, 4, 6, 7, 8, 9, 11], [1, 3, 4, 5, 6, 8, 10, 11], [1, 2, 3, 5, 7, 8, 10, 11], [4, 5, 7, 8, 10, 11], [1, 2, 6, 7, 8, 10, 11], [3, 4, 7, 9, 10, 11], [1, 5, 7, 9, 10, 11], [2, 3, 4, 5, 6, 7, 8, 13]]


  0%|          | 0/11 [00:00<?, ?it/s]

[[1, 2, 3, 4, 5, 7, 8, 9, 11],
 [1, 2, 3, 5, 7, 8, 10, 11],
 [3, 4, 7, 9, 10, 11]]

## 3 Dimensions

In [3]:
for n in tqdm(range(1, 43)):
    print(n, "-> ", end="")
    res = all_dissections(n, verbose=True)
    if res:
        for x in res:
            print(x[0])
            print(x[1])
            print(x[2])
            print("=" * 20)
    else:
        print("NO!")

  0%|          | 0/42 [00:00<?, ?it/s]

1 -> 

0it [00:00, ?it/s]

Candidate dissections: []


0it [00:00, ?it/s]

...

NO!
42 -> 

  0%|          | 0/41 [00:00<?, ?it/s]

Candidate dissections: []


0it [00:00, ?it/s]

NO!


## 2 Dimensions

In [4]:
for n in tqdm(range(1, 22)):
    print(n, "-> ", end="")
    res = all_dissections(n, 2, verbose=True)
    if res:
        for x in res:
            print(x[0])
            print(x[1])
            print(x[2])
            print("=" * 20)
    else:
        print("NO!")

  0%|          | 0/21 [00:00<?, ?it/s]

1 -> 

0it [00:00, ?it/s]

Candidate dissections: []


0it [00:00, ?it/s]

...

[1, 2, 3, 4, 5, 7, 8, 9, 11]
[[[ 6  8]
  [16 17]]

 [[ 8 11]
  [15 17]]

 [[ 8 11]
  [11 15]]

 [[ 6 11]
  [17 21]]

 [[ 0  6]
  [16 21]]

 [[ 0  8]
  [ 0  7]]

 [[ 0  8]
  [ 7 16]]

 [[11 20]
  [11 21]]

 [[ 8 20]
  [ 0 11]]]
None
[1, 2, 3, 5, 7, 8, 10, 11]
[[[11 12]
  [ 8 10]]

 [[12 15]
  [15 17]]

 [[12 15]
  [17 21]]

 [[15 20]
  [15 21]]

 [[12 20]
  [ 8 15]]

 [[11 20]
  [ 0  8]]

 [[ 0 11]
  [ 0 10]]

 [[ 0 12]
  [10 21]]]
None
[3, 4, 7, 9, 10, 11]
[[[ 0  3]
  [10 14]]

 [[ 3  8]
  [10 14]]

 [[ 0  8]
  [14 21]]

 [[ 0  9]
  [ 0 10]]

 [[ 9 20]
  [ 0 10]]

 [[ 8 20]
  [10 21]]]
None
21 -> 

  0%|          | 0/20 [00:00<?, ?it/s]

Candidate dissections: [[1, 2, 3, 4, 5, 6, 7, 8, 9, 11], [1, 2, 3, 5, 6, 7, 8, 10, 11], [4, 5, 6, 7, 8, 10, 11], [3, 4, 6, 7, 9, 10, 11], [1, 5, 6, 7, 9, 10, 11], [1, 2, 4, 5, 8, 9, 10, 11], [1, 7, 8, 9, 10, 11], [2, 3, 4, 5, 7, 8, 10, 12], [2, 4, 6, 7, 8, 10, 12], [1, 3, 4, 5, 6, 9, 10, 12], [1, 2, 3, 5, 7, 9, 10, 12], [4, 5, 7, 9, 10, 12], [1, 2, 6, 7, 9, 10, 12], [1, 3, 4, 8, 9, 10, 12], [3, 4, 5, 7, 8, 9, 13], [1, 2, 3, 6, 7, 8, 9, 13], [4, 6, 7, 8, 9, 13]]


  0%|          | 0/17 [00:00<?, ?it/s]

[1, 2, 3, 5, 7, 9, 10, 12]
[[[11 13]
  [ 9 10]]

 [[18 21]
  [20 22]]

 [[18 21]
  [16 20]]

 [[13 18]
  [16 22]]

 [[13 21]
  [ 9 16]]

 [[11 21]
  [ 0  9]]

 [[ 0 11]
  [ 0 10]]

 [[ 0 13]
  [10 22]]]
None
[1, 3, 4, 8, 9, 10, 12]
[[[ 8 10]
  [12 13]]

 [[ 0  3]
  [ 0  4]]

 [[ 3  8]
  [ 0  4]]

 [[ 0  8]
  [ 4 13]]

 [[ 0 10]
  [13 22]]

 [[10 21]
  [12 22]]

 [[ 8 21]
  [ 0 12]]]
None


## 4 Dimensions

In [5]:
for n in tqdm(range(1, 50)):
    print(n, "-> ", end="")
    res = all_dissections(n, 4, verbose=True)
    if res:
        for x in res:
            print(x[0])
            print(x[1])
            print(x[2])
            print("=" * 20)
    else:
        print("NO!")

  0%|          | 0/49 [00:00<?, ?it/s]

1 -> 

0it [00:00, ?it/s]

Candidate dissections: []


0it [00:00, ?it/s]

...

NO!
49 -> 

  0%|          | 0/48 [00:00<?, ?it/s]

Candidate dissections: []


0it [00:00, ?it/s]

NO!
